# 0. 原指导书中给出的示例代码

In [22]:
from elasticsearch.client import Elasticsearch
from datetime import datetime
es = Elasticsearch([{'host':'127.0.0.1','port':9200}], timeout=3600)
doc = {
    'author': 'Information Retrieval',
    'text': 'Test for ElasticSearch',
    'timestamp': datetime.now(),
}
if __name__ == "__main__":
    res = es.index(index="test-index", id=1, document=doc)
    #print(res['result']) 输出update
    res = es.get(index="test-index", id=1)
    print(res['_source'])  #输出原始内容
    es.indices.refresh(index="test-index")
    q1 = {
        "query": {
            "match":{
                "author" : "Information Retrieval"
            }
        }
    }
    res = es.search(index="test-index", body=q1)
    print("Got %d Hits:" % res['hits']['total']['value'])
    for hit in res['hits']['hits']:
        print("%(timestamp)s %(author)s: %(text)s" % hit["_source"])


D:\Anaconda3\lib\site-packages\elasticsearch\connection\base.py:209: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.15/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


{'author': 'Information Retrieval', 'text': 'Test for ElasticSearch', 'timestamp': '2021-11-12T18:00:07.867153'}
Got 1 Hits:
2021-11-12T18:00:07.867153 Information Retrieval: Test for ElasticSearch


<ipython-input-22-1a562ced9937>:22: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  res = es.search(index="test-index", body=q1)


# 1. 对少量邮件进行格式转化、索引构建

In [66]:
#一些简单测试
from email.parser import Parser
import os

def em2js(path,e_id): 
    """
    这一函数可以实现从email格式文件到json格式的转变和索引构建、文件插入
    返回json格式的信息jres
    
    path：需要构建索引的文件所在路径
    e_id：为邮件在索引中分配的id
    
    keyname：邮件的关键字列表，包括信息原始ID、发信时间、收发人等内容
    """
    keyname=['Message-ID','Date','From','To','Subject','Mime-Version','X-From','X-To','X-Folder','X-Origin','X-FileName']
    if os.path.exists(path):
        with open(path) as fp:
            em=fp.read()
            header=Parser().parsestr(em)# 将符合格式的email文件解析成email字典
            jres={}
            klen=len(keyname)
            for i in range(klen): # 按照keyname列表，转化为json
                item={keyname[i]:header[keyname[i]]}
                jres.update(item)
            es.index(index="em-index1", id=e_id, document=jres) #插入索引当中，索引名为em-index1
            es.indices.refresh(index="em-index1")
            return jres
    else:
        print("no file!")


es.indices.delete('index1')
jres1=em2js("maildir/allen-p/deleted_items/1",1)
jres2=em2js("maildir/allen-p/_sent_mail/4",2)
jres3=em2js("maildir/arora-h/all_documents/1",3)
jres4=em2js("maildir/badeer-r/all_documents/3",4)
#print("X-Cc:%s"%header['X-Cc'])



query = {
    "query": {
        "match_phrase":{
            "From" : "phillip.allen@enron.com"
        }#如果用match，则会把所有邮件都返回，因为es将点号视为分隔，所有含com的邮件都会返回
    }#如果用term，则会无法匹配任何邮件
}
params = {"ignore_unavailable": "true"}

res1 = es.search(index="index1", body=query,params=params)
print(res1)
print('\n')
for hit in res1['hits']['hits']:
    print("From: %(From)s  To:%(To)s ,%(X-cc)s" % hit["_source"],"id:%s"%hit["_id"])

<ipython-input-66-acaa55ac94e3>:23: DeprecationWarning: Using positional arguments for APIs is deprecated and will be disabled in 8.0.0. Instead use only keyword arguments for all APIs. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es.indices.delete('index1')


{'took': 2, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 1, 'relation': 'eq'}, 'max_score': 1.89712, 'hits': [{'_index': 'index1', '_type': '_doc', '_id': '2', '_score': 1.89712, '_source': {'Message-ID': '<19129466.1075855378242.JavaMail.evans@thyme>', 'Date': 'Thu, 10 May 2001 12:50:00 -0700 (PDT)', 'From': 'phillip.allen@enron.com', 'To': 'jsmith@austintx.com', 'Subject': '', 'Mime-Version': '1.0', 'Content-Type': 'text/plain; charset=us-ascii', 'Content-Transfer-Encoding': '7bit', 'X-From': 'Phillip K Allen', 'X-To': 'jsmith <jsmith@austintx.com>', 'X-cc': '', 'X-bcc': '', 'X-Folder': "\\Phillip_Allen_Jan2002_1\\Allen, Phillip K.\\'Sent Mail", 'X-Origin': 'Allen-P', 'X-FileName': 'pallen (Non-Privileged).pst'}}]}}


From: phillip.allen@enron.com  To:jsmith@austintx.com , id2


<ipython-input-66-acaa55ac94e3>:40: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  res1 = es.search(index="index1", body=query,params=params)


# 2. 核心代码：索引设计与构建
&emsp;这一部分的代码对上一小节实现的转化函数em2js进行了一定修改，实现了get_pa函数对整个maildir目录进行遍历，构建全部邮件的索引。

In [17]:
import os
from email.parser import Parser
from elasticsearch.client import Elasticsearch
from elasticsearch import helpers
from datetime import datetime

es = Elasticsearch([{'host':'127.0.0.1','port':9200}], timeout=3600)
#es.indices.delete('em-index1') 删除索引

def em2js(path,id_num): #本函数将Email转换为可以被es接受的json格式
    keyname=['Message-ID','Date','From','To','Subject','Mime-Version','X-From','X-To','X-Folder','X-Origin','X-FileName',"path"]
    # keyname：邮件中的一些关键信息的名称
    if os.path.exists(path):
        with open(path,errors='ignore') as fp: #忽略乱码的垃圾邮件
            em=fp.read()
            header=Parser().parsestr(em)
            jres={} #用来存储转换结果
            klen=len(keyname)
            for i in range(klen-1):
                item={keyname[i]:header[keyname[i]]}
                jres.update(item)
            item0={"path":path} # 把邮件的存储地址也存下来
            jres.update(item0)
            #es.index(index="em-index1", id=id_num, document=jres)
            #es.indices.refresh(index="em-index1")
            return jres
    else:
        print("no file!")
        

def get_pa(path,alldoc,nu,doc): 
    """
    这一函数实现对path路径下所有合法文件的遍历读取
    返回所有文件路径的列表alldoc
    
    path：待读目录所在路径
    nu：读取到的第nu个文件
    doc：邮件转化成的json文件集合
    """
    tot = os.listdir(path) #文件列表
    for pi in tot:
        tpath=os.path.join(path+"/",pi)
        if os.path.isfile(tpath):
            alldoc.append(tpath)#每找到一个文件，就把它的路径存下
            doc.append(em2js(tpath,nu)) #存下待建立索引的json文本
            nu+=1
        else:
            get_pa(tpath,alldoc,nu,doc)#遇到文件夹则继续递归
    return alldoc

pa=os.path.join("maildir")
id_num=1
doc0=[]
a=get_pa(pa,[],id_num,doc0) # a为所有文件路径
actions=[] 
#批量导入
for i in range(len(doc0)):
    action={'_op_type':'index',#操作参数可选：index插入文件 update更新索引 create建立索引 delete删除内容  
            '_index':'em-index1',#索引名称
            '_type':'doc',  #类型
            '_source':doc0[i]}
    actions.append(action)
helpers.bulk(client=es,actions=actions,index='em-index1')

(517401, [])

对建立好的索引进行一个简单的测试：

In [37]:
import os
from email.parser import Parser
from elasticsearch.client import Elasticsearch
from elasticsearch import helpers
from datetime import datetime

es = Elasticsearch([{'host':'127.0.0.1','port':9200}], timeout=3600)

query2 = {"query": 
          {"bool":
           {"must":
            [ {"match_phrase":{"From" : "eric.bass@enron.com"}},
             {"match_phrase":{"To" : "chance.rabon@enron.com"}},{"match_phrase":{"Subject" : "Rebooks"}}] }}
          ,"size":  10 
         }

params = {"ignore_unavailable": "true"}
res2 = es.search(index="em-index1", body=query2,params=params)
#print(res2)
for hit in res2['hits']['hits']:
    print("From: %(From)s  To:%(To)s ,%(path)s" % hit["_source"],"   id: %s"%hit["_id"])
print('\n')
print("Got %d Hits" % res2['hits']['total']['value'])
# es默认显示10条数据，在“size”处修改内容


From: eric.bass@enron.com  To:chance.rabon@enron.com ,maildir/bass-e/all_documents/1730    id: IW05HX0BgZQWTQ3PXK47
From: eric.bass@enron.com  To:chance.rabon@enron.com ,maildir/bass-e/all_documents/1761    id: Qm05HX0BgZQWTQ3PXK47
From: eric.bass@enron.com  To:chance.rabon@enron.com ,maildir/bass-e/discussion_threads/1784    id: W205HX0BgZQWTQ3PXbam
From: eric.bass@enron.com  To:chance.rabon@enron.com ,maildir/bass-e/discussion_threads/1794    id: Zm05HX0BgZQWTQ3PXbam
From: eric.bass@enron.com  To:chance.rabon@enron.com ,maildir/bass-e/sent/1050    id: 6m05HX0BgZQWTQ3PX71v
From: eric.bass@enron.com  To:chance.rabon@enron.com ,maildir/bass-e/sent/1081    id: DG05HX0BgZQWTQ3PX75v
From: eric.bass@enron.com  To:chance.rabon@enron.com ,maildir/bass-e/_sent_mail/1148    id: q205HX0BgZQWTQ3PYMT2
From: eric.bass@enron.com  To:chance.rabon@enron.com ,maildir/bass-e/_sent_mail/1179    id: y205HX0BgZQWTQ3PYMT2
From: eric.bass@enron.com  To:chance.rabon@enron.com ,maildir/bass-e/_sent_mail/37    

<ipython-input-37-1502a9859601>:18: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  res2 = es.search(index="em-index1", body=query2,params=params)


# 3.检索功能设计
&emsp;这部分代码实现了邮件检索的图形用户界面，可以按照收件人、发件人、标题等内容进行邮件检索。

In [39]:
import tkinter as tk
import tkinter.messagebox
 
m_dialog = tk.Tk()
m_dialog.minsize(350,100)# 初始大小为350×100
text1 = tk.Text(m_dialog,width=80) # 查询结果显示框

# 发件邮箱输入
fmFrom = tk.Frame(m_dialog)
fmFrom.pack(side="top")
txtFrom = tk.Label(fmFrom, text="发件邮箱 : ")
txtFrom.pack(side="left")
entryFrom = tk.Entry(fmFrom, bd=3,width=30)
entryFrom.pack(side="right")

# 收件邮箱输入
fmTo = tk.Frame(m_dialog)
fmTo.pack()
txtTo = tk.Label(fmTo, text="收件邮箱 : ")
txtTo.pack(side="left")
entryTo = tk.Entry(fmTo, bd=3,width=30)
entryTo.pack(side="right")

# 邮件主题输入
fmSbj = tk.Frame(m_dialog)
fmSbj.pack()
txtSbj = tk.Label(fmSbj, text="邮件标题 : ")
txtSbj.pack(side="left")
entrySbj = tk.Entry(fmSbj, bd=3,width=30)
entrySbj.pack(side="right")

# 邮件ID输入
fmID = tk.Frame(m_dialog)
fmID.pack(side="top")
txtID = tk.Label(fmID, text="邮件ID   : ")
txtID.pack(side="left")
entryID = tk.Entry(fmID, bd=3,width=30)
entryID.pack(side="right")
"""

eric.bass@enron.com

chance.rabon@enron.com

Rebooks

"""
def searchm():
    text1.delete(0.0,"end")
    if entryFrom.get() == "" and entryTo.get() == "" and entrySbj.get() == "" and entryID.get()=="":
        print(tkinter.messagebox.showinfo("Result", "No message."))  # 未输入查询信息时进行提醒
    else:
        fro0 = entryFrom.get() # 发件人邮箱
        to0 = entryTo.get() # 收件人邮箱
        sbj0 = entrySbj.get() # 邮件主题
        id0=entryID.get() #邮件的Message-ID
        
        # 若对应查询栏为空，则忽略这一信息的约束，只对其他内容进行查询
        query = {"query": {"match_phrase":{"From" : fro0}},"size":  100}
        
        if (fro0==""and to0!=""and sbj0==""):  # 仅指定了收件人
            query = {"query": {"match_phrase":{"To" : to0}},"size":  100} 
        if (fro0==""and to0==""and sbj0!=""):  # 仅指定了邮件主题
            query = {"query": {"match_phrase":{"Subject" : sbj0}},"size":  100} 
        if (fro0!=""and to0!=""and sbj0==""):  # 仅指定了收发人
            query = {"query": {"bool":{"must":[ {"match_phrase":{"From" : fro0}},{"match_phrase":{"To" : to0}} ] }},"size":  100}
        if (fro0!=""and to0==""and sbj0!=""):  # 仅指定了发件人和主题
            query = {"query": {"bool":{"must":[ {"match_phrase":{"From" : fro0}},{"match_phrase":{"Subject" : sbj0}} ] }},"size":  100}
        if (fro0==""and to0!=""and sbj0!=""):  
            query = {"query": {"bool":{"must":[ {"match_phrase":{"Subject" : sbj0}},{"match_phrase":{"To" : to0}} ] }},"size":  100}
        if (fro0!=""and to0!=""and sbj0!=""):
            query = {"query": {"bool":{"must":[ {"match_phrase":{"From" : fro0}},{"match_phrase":{"To" : to0}},{"match_phrase":{"Subject" : sbj0}} ] }},"size":  100}
        if(id0!=""): # 只要指定了ID，就不再需要其他信息，ID可以唯一确定一封邮件
            query = {"query": {"match_phrase":{"Message-ID" : id0}},"size":  100}
        
        params = {"ignore_unavailable": "true"}
        res2 = es.search(index="em-index1", body=query,params=params)
        num = 0
        for hit in res2['hits']['hits']: #显示查询结果
            num += 1
            text1.insert("insert", ("Some messages about mail-%d"%num+":\n  ID: %(Message-ID)s\n  From: %(From)s   To:%(To)s \n  path_in_folder: %(path)s"%hit["_source"]))
            text1.insert("insert","\n\n")
        text1.insert("end", "")
        text1.pack()


button = tk.Frame(m_dialog)
button.pack(side="bottom") # 添加一个search按钮
searchb = tk.Button(
    button,
    text="search",
    activeforeground="white",
    activebackground="black",
    width="15",
    command=searchm
)
searchb.pack(side="left")

m_dialog.mainloop()


<ipython-input-39-20cbf3c9c197>:70: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  res2 = es.search(index="em-index1", body=query,params=params)
